In [29]:
# %% [markdown]
# # Strategic Workforce Analysis: AI Integration vs. Structural Risk (2010-2025)
# ## Task 1: Data Preprocessing & Outlier Handling

# %%
import pandas as pd
import numpy as np
import os

# --- 1. Robust Path Setup ---
# Get current working directory (notebooks) and move to parent to find data/
BASE_DIR = os.path.dirname(os.getcwd())
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', 'raw', 'ai_impact_jobs_2010_2025.csv')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

# Ensure folder exists
os.makedirs(PROCESSED_DIR, exist_ok=True)

# 2. Load Dataset
df = pd.read_csv(RAW_DATA_PATH)

# 3. Data Cleaning (Nulls)
df['ai_skills'] = df['ai_skills'].fillna('Not Specified')
df['ai_keywords'] = df['ai_keywords'].fillna('None')

# 4. NESTED IQR LOGIC (The "Flower" of the project)
# We calculate IQR for each (Region + Seniority) group to remove local anomalies.
cleaned_chunks = []

# Get all unique combinations of Region and Seniority
for region in df['region'].unique():
    for level in df['seniority_level'].unique():
        # Create a subset
        subset = df[(df['region'] == region) & (df['seniority_level'] == level)]
        
        if len(subset) > 3: # Only clean if we have enough data
            q1 = subset['salary_usd'].quantile(0.25)
            q3 = subset['salary_usd'].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            # Filter the subset
            subset = subset[(subset['salary_usd'] >= lower) & (subset['salary_usd'] <= upper)]
        
        cleaned_chunks.append(subset)

# Combine everything back - Columns are guaranteed to stay
df_cleaned = pd.concat(cleaned_chunks).reset_index(drop=True)

# 5. Save the Cleaned Dataset
CLEANED_FILE_PATH = os.path.join(PROCESSED_DIR, 'ai_impact_jobs_cleaned.csv')
df_cleaned.to_csv(CLEANED_FILE_PATH, index=False)

print(f"Task 1 Success! Cleaned data saved at: {CLEANED_FILE_PATH}")
print(f"Final Columns Check: {df_cleaned.columns.tolist()}")

Task 1 Success! Cleaned data saved at: /Users/miraekang/proyectos/eda/data/processed/ai_impact_jobs_cleaned.csv
Final Columns Check: ['job_id', 'posting_year', 'country', 'region', 'city', 'company_name', 'company_size', 'industry', 'job_title', 'seniority_level', 'ai_mentioned', 'ai_keywords', 'ai_intensity_score', 'core_skills', 'ai_skills', 'salary_usd', 'salary_change_vs_prev_year_percent', 'automation_risk_score', 'reskilling_required', 'ai_job_displacement_risk', 'job_description_embedding_cluster', 'industry_ai_adoption_stage']
